In [ ]:
%pip install bs4

In [ ]:
from bs4 import BeautifulSoup
import requests

URL = "http://www.wikipedia.org"
response = requests.get(URL)
print(response.content)
print(response.text)
print(response.status_code)
response.raise_for_status()


In [ ]:
soup = BeautifulSoup(response.text, "html.parser")
print(soup.prettify())

print(soup.title)
print(soup.title.text)
print(soup.title.get_text()[0::1])

print(soup.h1)
print(soup.h1.parent.name)

print(soup.body.h1.previous_sibling)
print(soup.body.h1.attrs['class'][0])

print(soup.find_all("a"))
print(soup.find(class_="link-box"))
print(list(soup.body.div.stripped_strings))

In [ ]:
import re

def returnMatches(target):
    return re.findall("[a-zA-Z]+", target)


print(returnMatches(soup.body.div.text))


In [ ]:
print(soup.select("h1 > span")[0])

In [ ]:
import pickle
import os

filename = "wikipedia-scraped-data.pickle"
if os.path.exists(filename):
    with open(filename, 'rb') as f:
        response = pickle.load(f)

else: 
    result = requests.get(URL)
    with open(filename, "wb") as f:
        pickle.dump(result, f)


print(result.text[0:100])

In [ ]:
import csv

# row = [{"First Name": "Ibrahim", "Last Name": "Rehman", "Age": 20}, {"First Name": "Usman", "Last Name": "Naveed", "Age": 20}, {"First Name": "Fasihullah", "Last Name": "Ejaz", "Age": 20}]



URL = "https://en.wikipedia.org/wiki/Main_Page"
response = requests.get(URL)
soup = BeautifulSoup(response.text, "html.parser")
links = soup.select("#mp-other-content > ul > li a")
linksArr = []
for anchors in links:
    linksArr.append(anchors.text)
linksDes = soup.select("#mp-other-content > ul > li")
linksDesArr = []
for linkDes in linksDes:
    linksDesArr.append(linkDes.text.split("– ")[-1])


rowHead = ["Links", "Links Description"]

rows = []

for i in range(len(linksArr)):
    rows.append({"Links": linksArr[i], "Links Description": linksDesArr[i]})


with open("wikipedia-scraped-data.csv", "w") as f:
    writer = csv.DictWriter(f, fieldnames=rowHead)
    writer.writeheader()
    writer.writerows(rows)

In [ ]:
import pandas as pd

df_raw = pd.read_csv("wikipedia-scraped-data.csv")
df = df_raw.copy()
# df.head()
# df.shape
# df.sample(2)
# df.describe()


In [ ]:
df.links.value_counts()

In [ ]:
df.Links.value_counts().sort_index()

In [ ]:
%pip install seaborn

In [ ]:
import seaborn as sns

ax = sns.countplot(data=df, y='Links Description')
ax.set_title("Count Plot")


In [ ]:
df.isnull().sum()

In [ ]:
df = df.query("Links != 'Community portal' and Links != 'Village pump'").copy()
df.head()

df['Links Author'] = df.Links.apply(lambda link: f"{len(link)} Links")

df.head()


In [ ]:
ax = df.Links.hist()
ax.set_xlabel("Links")
ax.set_ylabel("Frequency")
ax.set_title("Links Histogram")

In [ ]:
df.Links.str.upper()